# Module 8: 4 - Optimizing RAG - Knowledge Graph using Neo4j
----------------------------------------------------------------------------
In this lesson, we will explore how to upload knowledge graph context to a Neo4j server using the [cypher](https://neo4j.com/docs/cypher-manual/current/introduction/) clarative query language.

## Objectives
* Upload GraphRAG context to Neo4j server
* Learn cypher statements schema to upload GraphRAG context
* Visualize GraphRAG context in Ne4j server

## What this session covers:
* Create Neo4j driver
* Define function to upload GraphRAG context to Neo4j
* Read GraphRAG context from parquet files
* Upload GraphRAG context to Neo4j server

## Pre-Requisites
* Neo4j database server running

## Reference
* https://github.com/graphrag/ms-graphrag/blob/graphrag_import_neo4j_cypher/examples_notebooks/neo4j/graphrag_import_neo4j_cypher.ipynb

## Install Libraries

In [ ]:
#!pip install pandas
#!pip install neo4j
#!pip install time

## Define Paths

In [ ]:
# Defining path to artifacts
GRAPHRAG_FOLDER="./data/output/20240801-133147/artifacts"

## Start Neo4j Docker Container

docker run \
    --restart always \
    --publish=7474:7474 --publish=7687:7687 \
    --env NEO4J_AUTH=neo4j/pandaneo4j \
    --volume=neo4j-data \
    --name neo4j-apoc \
    --env NEO4J_apoc_export_file_enabled=true \           
    --env NEO4J_apoc_import_file_enabled=true \           
    --env NEO4J_apoc_import_file_use__neo4j__config=true \
    --env NEO4J_PLUGINS=\[\"apoc\"\] \
    neo4j:latest

## Connect to Neo4j Server

In [ ]:
from neo4j import GraphDatabase

# Defining Neo4J connection parameters
NEO4J_URI="neo4j://localhost:7687" # or neo4j+s://xxxx.databases.neo4j.io 
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="pandaneo4j"
NEO4J_DATABASE="neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## Define Function to Send Data to Neo4j

In [1]:
import pandas as pd
import time

# Defining function 
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, 
                df is the dataframe to import, 
                and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

# Define Constraints (Avoid duplciates)

In [ ]:
# Create constraints, idempotent operation
statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

# Uploading constraints
for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Uploading Knowledge Graph Context

### Documents (.md files)

In [ ]:
# Create documents dataframe
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet', columns=["id", "title"])
doc_df.head()

,id,title
0,afd6025b421d49a061e3a6acf74daedd,FIN7.md
1,2362aa068c9293d37a7f821c903e629c,Sandworm_Team.md
2,f9ba917021b741eb0f3f9f29a48196e1,Carbanak.md
3,5767232d0a7dd9885f8d1b8c2ef28d3e,Turla.md
4,fd55c2bb3647d332545d696deb751d13,APT29.md


In [ ]:
# Define and upload context
doc_statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""
batched_import(doc_statement, doc_df)

{'_contains_updates': True, 'labels_added': 9, 'nodes_created': 9, 'properties_set': 18}
9 rows in 0.18384718894958496 s.


9

### Text Units (Chunks)

In [ ]:
# Create chunks dataframe
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet',
                          columns=["id","text","n_tokens","document_ids"])
text_df.head()

,id,text,n_tokens,document_ids
0,bdbda95d8a567d3c9ee1d96ffbd1e2c0,# Sandworm Team - G0034\n\n**Created**: 2017-05-31T21:32:04.588Z\n\n**Modified**: 2024-04-06T19:...,1200,[2362aa068c9293d37a7f821c903e629c]
1,5e839788284eecb63814ac126969a91f,HTML tags for the communication traffic between the C2 server.(Citation: ESET Telebots Dec 2016...,1200,[2362aa068c9293d37a7f821c903e629c]
2,51659aba82df667a55ca598ef7d761f4,door password to maintain persistence within the target network. [Sandworm Team](https://attack....,1200,[2362aa068c9293d37a7f821c903e629c]
3,e8027f448bcfaa176bfb6c261f96b671,://attack.mitre.org/software/S1058) operations.(Citation: US District Court Indictment GRU Unit ...,1200,[2362aa068c9293d37a7f821c903e629c]
4,2dc4bd2cb621b71f05523705af6b571f,1505.003|Web Shell|[Sandworm Team](https://attack.mitre.org/groups/G0034) has used webshells inc...,1200,[2362aa068c9293d37a7f821c903e629c]


In [ ]:
# Define and upload context
text_statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""
batched_import(text_statement, text_df)

{'_contains_updates': True, 'labels_added': 46, 'relationships_created': 46, 'nodes_created': 46, 'properties_set': 138}
46 rows in 0.1977369785308838 s.


46

### Entities

In [ ]:
# Create entities dataframe
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet',
                            columns=["name","type","description","human_readable_id","id","description_embedding","text_unit_ids"])
entity_df.head(n = 2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,"""SANDWORM TEAM""","""ORGANIZATION""","Sandworm Team, also known by multiple aliases including ELECTRUM, Telebots, IRON VIKING, BlackEn...",0,b45241d70f0e43fca764df95b2b81f77,"[0.022811800241470337, 0.04036780819296837, 0.04281684383749962, 0.04548601433634758, -0.0054896...","[2dc4bd2cb621b71f05523705af6b571f, 396c22047983755b945f5235333819d3, 51659aba82df667a55ca598ef7d..."
1,"""RUSSIA'S GENERAL STAFF MAIN INTELLIGENCE DIRECTORATE (GRU) MAIN CENTER FOR SPECIAL TECHNOLOGIES...","""ORGANIZATION""","""Russia's GRU GTsST, military unit 74455, is the organization behind Sandworm Team, involved in ...",1,4119fd06010c494caa07f439b333f4c5,"[-0.03776426985859871, 0.02152342163026333, 0.05562674254179001, 0.02057747170329094, -0.0243981...",[bdbda95d8a567d3c9ee1d96ffbd1e2c0]


In [ ]:
# Define and upload context
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:EXTRACTED_FROM]->(e)
"""
batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 357, 'relationships_created': 620, 'nodes_created': 357, 'properties_set': 1428}
357 rows in 0.8195133209228516 s.


357

### Relationships

In [ ]:
# Create relationships dataframe
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet',
                         columns=["source","target","id","rank","weight","human_readable_id","description","text_unit_ids"])
rel_df.head(n = 3)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,"""SANDWORM TEAM""","""RUSSIA'S GENERAL STAFF MAIN INTELLIGENCE DIRECTORATE (GRU) MAIN CENTER FOR SPECIAL TECHNOLOGIES...",36be44627ece444284f9e759b8cd25c6,43,1.0,0,"""Sandworm Team is attributed to and operates under the GRU GTsST military unit 74455.""",[bdbda95d8a567d3c9ee1d96ffbd1e2c0]
1,"""SANDWORM TEAM""","""GRU UNIT 74455""",a64b4b17b07a44e4b1ac33580d811936,43,2.0,1,"Sandworm Team, directly associated with GRU Unit 74455, conducts cyber operations under its aegi...","[bdbda95d8a567d3c9ee1d96ffbd1e2c0, e8027f448bcfaa176bfb6c261f96b671]"
2,"""SANDWORM TEAM""","""GRU UNIT 26165""",423b72bbd56f4caa98f3328202c1c3c9,43,1.0,2,"""Sandworm Team collaborated with GRU Unit 26165, also known as APT28, in cyber operations.""",[bdbda95d8a567d3c9ee1d96ffbd1e2c0]


In [ ]:
# Define and upload context
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""
batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 285, 'properties_set': 1710}
285 rows in 0.27146220207214355 s.


285

### Communities

In [ ]:
# Create communities dataframe
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet', 
                     columns=["id","level","title","text_unit_ids","relationship_ids"])
community_df.head(n=5)

,id,level,title,text_unit_ids,relationship_ids
0,0,0,Community 0,"[2dc4bd2cb621b71f05523705af6b571f,396c22047983755b945f5235333819d3,51659aba82df667a55ca598ef7d76...","[36be44627ece444284f9e759b8cd25c6, a64b4b17b07a44e4b1ac33580d811936, 423b72bbd56f4caa98f3328202c..."
1,4,0,Community 4,"[202bcc7099b468087583d88e9ddb57de,396c22047983755b945f5235333819d3,4e30c37f12c9daf946a2010e949a6...","[30a251bc3d04430d82b5a1a98c7b8c75, 93e1d19f9bfa4c6b8962d56d10ea9483, 8046335ba70b434aa3188392a74..."
2,7,0,Community 7,"[202bcc7099b468087583d88e9ddb57de,396c22047983755b945f5235333819d3,4e30c37f12c9daf946a2010e949a6...","[f026fab8fec948ae9e7baa2ad715e6ef, d0d7ed36d6f54b5d986dfd854096b728, bf6a4c18f44042799eb7456a6b8..."
3,3,0,Community 3,"[5e839788284eecb63814ac126969a91f,5f65c33bcf3ac888e54026ea0984d754,6e70fdd609af24bbd9ceb18ada071...","[0211d61aae834229a3a1e004ff5cc658, ccbbbcc055c34709abcf103208c2c299, 989add81cf874018a569239b68d..."
4,5,0,Community 5,"[5e839788284eecb63814ac126969a91f,6e70fdd609af24bbd9ceb18ada07107b,6f508e42f90f87ca9825acb2e42e4...","[711ba818354546cea69f1532b92a2f26, 5c4d8a8f9c104176b87d2bfdf04ae0bd, 5a781604f1fb4719b730f43f534..."


In [ ]:
# Define and upload context
community_statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""
batched_import(community_statement, community_df)

{'_contains_updates': True, 'labels_added': 15, 'relationships_created': 470, 'nodes_created': 15, 'properties_set': 45}
15 rows in 0.17423105239868164 s.


15

### Community Reports

In [ ]:
# Create community reports dataframe
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet',
                               columns=["id","community","level","title","summary", "findings","rank","rank_explanation","full_content"])
community_report_df.head(n = 3)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,1bd35e5c-af30-4e33-b898-a8d58ef37083,10,1,Carbanak Group and Carbanak Malware,"The community focuses on the Carbanak Group, known for its use of Carbanak malware, highlighting...","[{'explanation': 'The Carbanak Group is a prominent entity within the cybercrime community, know...",8.5,The high impact severity rating reflects the significant threat posed by the Carbanak Group's ac...,"# Carbanak Group and Carbanak Malware\n\nThe community focuses on the Carbanak Group, known for ..."
1,4b8eb4db-bf82-4576-920a-ebf15d72fb95,11,1,Turla Cyber Espionage Activities and Infrastructure Targets,"This report delves into the sophisticated cyber espionage activities of the Turla group, focusin...","[{'explanation': 'Turla, a group with ties to Russia's FSB, has been active since at least 2004,...",8.5,The high impact severity rating reflects Turla's sophisticated capabilities and strategic target...,# Turla Cyber Espionage Activities and Infrastructure Targets\n\nThis report delves into the sop...
2,0f7311be-7f8a-4e9e-bf52-a8eba7944943,12,1,Gazer Malware and Turla Network,"The community focuses on the Gazer malware, its technical mechanisms such as mutex names and nam...","[{'explanation': 'Gazer malware, associated with the Turla group, employs advanced obfuscation t...",8.5,The high impact severity rating reflects the sophisticated obfuscation and communication strateg...,"# Gazer Malware and Turla Network\n\nThe community focuses on the Gazer malware, its technical m..."


In [ ]:
# Define and upload context
community_report_statement = """
Merge (c:__Community__ {community:value.community})
SET c += value {.community, .level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_report_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 73, 'relationships_created': 73, 'nodes_created': 73, 'properties_set': 324}
15 rows in 0.2107398509979248 s.


15